## Error and Attack Robustness of Time Variant Markovian Networks

In [2]:
import networkx as nx
import numpy as np
import random
import math
import matplotlib.pyplot as plt

In [5]:
class temporal_markov:
    """ handles all functional requirements of a temporal markovian graph """
    def __init__(self, p, q, num_nodes, average_degree, tau):
        self.p = p
        self.q = q
        self.N = num_nodes
        self.average_degree = average_degree
        self.initial_edge_probability = average_degree / (self.N - 1)
        self.current_timestep = 0
        self.Graphs = []    
        self.tau = tau
        self.initialise_graph()
        
    def initialise_graph(self):
        G = nx.erdos_renyi_graph(self.N, self.initial_edge_probability)
        self.Graphs.append(G)
    
    def update_tick(self):
        size = len(self.Graphs)
        G = self.Graphs[size - 1]
        adj_matrice = np.array(nx.adjacency_matrix(self.G))
        l,b = self.adj_matrice
        for i in range(l):
            for j in range(b):
                toss = np.random.uniform(0,1)
                if adj_matrice[i,j] == 1:
                    if toss > self.P :
                        adj_matrice[i,j] = 0
                elif adj_matrice[i,j] == 0:
                    if toss > self.q :
                        adj_matrice[i,j] = 1
        self.Graphs.append(nx.from_numpy_matrix(adj_matrice))
        self.current_timestep += 1
        
    
    def compute_average_degree(self):
        avg_degrees = [0 for i in range(self.N)]
        for t in range(self.tau):
            g = self.Graphs[t]
            degrees = g.degree()
            for vertx in range(self.N):
                deg = degrees[vertx]
                avg_degrees[vertx] += deg
        for vx in range(self.N):
            avg_degrees[vx] = avg_degrees[vx] / (self.N - 1)
            
        
    
    def compute_temporal_distances_and_efficiency(self):
        self.temporal_distances = self.zeros((self.N,self.N,self.tau, self.tau))
        #td[i,j,t1,t2] = 1 if there exist a temporal path between i,j of size t2 - t1 in the time period t1 and t2 
        
        #initialize all value with inf 
        inf = 2 * self.N
        self.temporal_distances += inf
        
        #now for all values td[i,i,t1,t1] = 0 because you can reach to yourself in 0 steps
        for i in range(self.N):
            for j in range(self.tau):
                self.temporal_distances[i,i,j,j] = 0
        
        for start_time in range(self.tau):
            for end_time in range(start_time + 1, self.tau):
                #get the adjacency for the current matrix 
                current_adj = nx.adjacency_matrix(self.Graphs[end_time])
                for middle_node in range(self.N):
                    for end_node in range(self.N):
                        if current_adj[middle_node, end_node] == 1:
                            for start_node in range(self.N):
                                if self.temporal_distances[start_node, middle_node, start_time, end_time -1] == 1:
                                    #you can reach from s to e in end_time - start_time only if there exist a middle such that you can reach from s 
                                    #in ((end_time - 1) - start_time) such that there a link between middel and end
                                    self.temporal_distances[start_node,end_node, start_time, end_time] = 1
        
        self.temporal_scores = np.zeros((self.N,self.N)) + inf
        for node in range(self.N):
            self.temporal_scores[node,node] = 0
        temporal_efficiency = 0
        temporal_closeness = np.zeros((self.N))
        for start_node in range(self.N):
            for end_node in range(self.N):
                for start_time in range(self.tau):
                    for end_time in range(self.tau):
                        if self.temporal_distances[start_node,end_node,start_time, end_time]:
                            self.temporal_scores[start_node, end_node] = min(self.temporal_scores, end_time - start_time)
                if start_node != end_node:
                    if self.temporal_scores[start_node, end_node] != inf:
                        temporal_efficiency += 1 / self.temporal_scores[start_node, end_node]
                    temporal_closeness[start_node] += self.temporal_scores[start_node, end_node]
        
        temporal_closeness = temporal_closeness / (self.N - 1)
        
        
        
    

        temporal_efficiency = temporal_efficiency / (self.N * (self.N - 1))
        return temporal_efficiency, temporal_closeness
    
                    
    
            
    
    
    
        
        

In [10]:
r = nx.erdos_renyi_graph(100, 0.2)
s = r.degree()
print(s)


[(0, 19), (1, 22), (2, 26), (3, 21), (4, 25), (5, 19), (6, 23), (7, 15), (8, 20), (9, 19), (10, 23), (11, 20), (12, 19), (13, 20), (14, 25), (15, 15), (16, 24), (17, 23), (18, 22), (19, 20), (20, 22), (21, 20), (22, 17), (23, 16), (24, 18), (25, 15), (26, 17), (27, 21), (28, 18), (29, 14), (30, 24), (31, 21), (32, 18), (33, 24), (34, 18), (35, 22), (36, 21), (37, 19), (38, 27), (39, 19), (40, 22), (41, 14), (42, 20), (43, 21), (44, 16), (45, 18), (46, 20), (47, 16), (48, 22), (49, 13), (50, 22), (51, 21), (52, 17), (53, 15), (54, 25), (55, 19), (56, 17), (57, 22), (58, 21), (59, 16), (60, 28), (61, 18), (62, 23), (63, 22), (64, 18), (65, 21), (66, 11), (67, 18), (68, 19), (69, 21), (70, 18), (71, 21), (72, 16), (73, 24), (74, 24), (75, 22), (76, 17), (77, 19), (78, 20), (79, 25), (80, 22), (81, 17), (82, 23), (83, 18), (84, 19), (85, 21), (86, 16), (87, 21), (88, 17), (89, 21), (90, 17), (91, 27), (92, 25), (93, 31), (94, 16), (95, 16), (96, 20), (97, 20), (98, 25), (99, 19)]


In [14]:
print(s[0], s[1], s[2], s[10],s[98])

19 22 26 23 25
